In [1]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')


# Load environment variables from the .env file
load_dotenv()

True

In [15]:
def elastic_search(es_client, query, limit=3, filter=None, random=False):
    if random:
        search_query = {
          "size": 10,
          "query": {
            "function_score": {
              "functions": [
                {
                  "random_score": {}
                }
              ]
            }
          }
        }
    else:
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["relief", "positive_effects", "flavours"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {  # TODO: add filter as parameter
                        "term": {
                            "category": "flower"
                        }
                    }
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    
    return response['hits']['hits'][:limit]

def pretty(search_results):
    result_docs = []
    
    for hit in search_results:
        result_docs.append({hit['_score']: hit['_source']})
    return result_docs


query = 'Lemon, headache'
index_name = "course-questions"

res = elastic_search(es_client, query, random=True)
sample = res[0]['_source']
print(sample.keys())
print()
print(pretty(res))

dict_keys(['title', 'tags', 'relief', 'positive_effects', 'flavours', 'avg_rating', 'num_ratings'])

[{0.9979122: {'title': 'White Widow', 'tags': 'Sativa Dominant|60% Sativa/40% Indica', 'relief': 'bipolar disorder chronic pain depression fatigue headaches migraines muscle spasms ptsd', 'positive_effects': 'creative euphoria happy uplifting', 'flavours': 'dank pine spicy sweet woody', 'avg_rating': 4.4, 'num_ratings': 112}}, {0.9906636: {'title': 'Pink Kush', 'tags': 'Indica Dominant|90% Indica/10% Sativa', 'relief': 'anxiety bipolar disorder chronic pain depression inflammation insomnia loss of appetite migraines ptsd stress', 'positive_effects': 'body high euphoria happy hungry relaxing sleepy', 'flavours': 'berry candy sweet vanilla', 'avg_rating': 4.7, 'num_ratings': 50}}, {0.9890757: {'title': 'Lemon Kush', 'tags': 'Hybrid,|50%/50%', 'relief': 'bipolar disorder chronic pain depression headaches insomnia migraines ptsd stress', 'positive_effects': 'creative euphoria happy relaxing

In [34]:
import pandas as pd

current_file_dir = os.getcwd()
app_files_dir = os.path.join(os.path.dirname(current_file_dir), 'data', 'pipelines-data')

print(os.listdir(app_files_dir))

res_csv_path = os.path.join(app_files_dir, 'content_green_bro.csv')

content_db = pd.read_csv(res_csv_path)

['content_reviews_green_bro.json', 'content_green_bro.csv']


In [33]:
import json

def load_reviews(reviews_file_path):
    with open(reviews_file_path, 'r') as f:
        reviews_dict = json.load(f)
    return reviews_dict

reviews_dict = load_reviews(os.path.join(app_files_dir, 'content_reviews_green_bro.json'))
print(reviews_dict[0])

{'item': '_marijuana-strains_indica-dominant-hybrid_fire-og.html', 'review': " All I gotta say is, make sure all ya chores are done, cuz it's hitting fast and hard...I don't feel like doing nothing.Very muscle relaxing as well. ", 'item_name': 'fire-og'}


In [32]:
def get_item_reviews(item_name):
    reviews = [i['review'] for i in reviews_dict if i['item_name'] == item_name]
    return reviews

get_item_reviews('fire-og')

[" All I gotta say is, make sure all ya chores are done, cuz it's hitting fast and hard...I don't feel like doing nothing.Very muscle relaxing as well. ",
 ' Fire OG is classic OG taste meets classic OG feels. At first it’s smells like Skywalker OG, but the taste and feel is calm ride on a cosmic surfboard. Definitely pick it up and keep it stocked up for those evenings you need a little extra kushy cloud vibes. ',
 ' Was expecting a little more with this one with all the hype around it. Got a nice batch testing 28% thc and 2+ % terps. Dense, dark green buds with the signature small red pistils. Nice coating and glimmer of the trichomes. The taste and smell are in the forest og spectrum. The high comes on quickly with an uplift of some euphoria and a lil energy . Over all I was expecting some more oomph and distinguishability. It’s a nice OG but not up there with the best imho. ',
 ' Works great for nausea ,appetite and some pain due to cancer...Makes chemo somewhat easier to deal with